In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors as cmod
import subprocess as sub

from scipy.integrate import ode

from scipy.interpolate import interp2d

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.animation as animation
from IPython.display import HTML

import matplotlib.patches as mpatches


from matplotlib.animation import PillowWriter  


from scipy.spatial.transform import Rotation as R

from geofuns import *
from FantasyWorld import *
from plotting import *
from demographics import *

import pickle

In [7]:
with open("DemoWorld/world.p", 'rb') as fl:
    world = pickle.load(fl)
with open("DemoWorld/demographics.p", 'rb') as fl:
    demos = pickle.load(fl)